In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [155]:
df = pd.read_csv('train.csv')

In [156]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0


In [157]:
df["risk_factors"] = df[[
        "avg_glucose_level", "age", "bmi",
        "hypertension", "heart_disease",
        "smoking_status"
    ]].apply(
        lambda x: \
        0 + (1 if x.avg_glucose_level > 99 else 0) + \
        (1 if x.age > 45 else 0) + (1 if x.bmi > 24.99 else 0) + \
        (1 if x.hypertension == 1 else 0) + \
        (1 if x.heart_disease == 1 else 0) + \
        (1 if x.smoking_status in ["formerly smoked", "smokes"] else 0),
        axis=1
    )



In [158]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,risk_factors
0,0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0,1
1,1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0,1
2,2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0,2
3,3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0,2
4,4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0,1


In [159]:
df = pd.get_dummies(df,columns=['work_type','smoking_status'])
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,risk_factors,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0,Male,28.0,0,0,Yes,Urban,79.53,31.1,0,1,0,0,1,0,0,0,0,1,0
1,1,Male,33.0,0,0,Yes,Rural,78.44,23.9,0,1,0,0,1,0,0,0,1,0,0
2,2,Female,42.0,0,0,Yes,Rural,103.00,40.3,0,2,0,0,1,0,0,1,0,0,0
3,3,Male,56.0,0,0,Yes,Urban,64.87,28.8,0,2,0,0,1,0,0,0,0,1,0
4,4,Female,24.0,0,0,No,Rural,73.36,28.8,0,1,0,0,1,0,0,0,0,1,0


In [160]:
X = df.drop(['id','stroke','gender','work_type_Never_worked','ever_married','Residence_type','smoking_status_Unknown','work_type_children'],axis=1)
X.columns

Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'risk_factors', 'work_type_Govt_job', 'work_type_Private',
       'work_type_Self-employed', 'smoking_status_formerly smoked',
       'smoking_status_never smoked', 'smoking_status_smokes'],
      dtype='object')

In [181]:
X=df[['work_type_Private','work_type_Govt_job','work_type_Self-employed','risk_factors']]

In [182]:
y = df['stroke']

In [183]:
len(X.columns)

4

In [184]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state =4)


In [185]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform((X))
X_test = sc.transform(X_test)


In [186]:
from keras.utils import np_utils
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255



In [187]:
Y_train = np_utils.to_categorical(y_train, 26)
Y_test = np_utils.to_categorical(y_test, 26)


In [188]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [189]:
ann = Sequential()
ann.add(Dense(units=4, activation='relu'))
ann.add(Dense(units=3, activation='relu'))

ann.add(Dense(units=1, activation='softplus'))


opt = tf.keras.optimizers.Adam()
ann.compile(loss="binary_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

In [190]:
ann.fit(X_train,y_train,batch_size=32,validation_data=(X_test,y_test),epochs=50)

Epoch 1/50
383/383 [==============================] - 3s 4ms/step - loss: 0.7270 - accuracy: 0.4817 - val_loss: 0.3528 - val_accuracy: 0.9680
Epoch 2/50
383/383 [==============================] - 1s 3ms/step - loss: 0.2431 - accuracy: 0.9564 - val_loss: 0.1644 - val_accuracy: 0.9680
Epoch 3/50
383/383 [==============================] - 1s 3ms/step - loss: 0.1834 - accuracy: 0.9564 - val_loss: 0.1468 - val_accuracy: 0.9680
Epoch 4/50
383/383 [==============================] - 1s 3ms/step - loss: 0.1802 - accuracy: 0.9564 - val_loss: 0.1449 - val_accuracy: 0.9680
Epoch 5/50
383/383 [==============================] - 1s 3ms/step - loss: 0.1799 - accuracy: 0.9564 - val_loss: 0.1443 - val_accuracy: 0.9680
Epoch 6/50
383/383 [==============================] - 1s 3ms/step - loss: 0.1799 - accuracy: 0.9564 - val_loss: 0.1441 - val_accuracy: 0.9680
Epoch 7/50
383/383 [==============================] - 1s 3ms/step - loss: 0.1797 - accuracy: 0.9564 - val_loss: 0.1437 - val_accuracy: 0.9680
Epoch 

In [191]:
ann.evaluate(X_test, y_test)
y_pred = ann.predict(X_test)


96/96 [==============================] - 0s 3ms/step


In [192]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [193]:
cm

NameError: name 'cm' is not defined

In [194]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=1000)

In [195]:
clf.score(X_test,y_test)


c:\users\sagar\envs\test\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


0.967984318850049

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
X = pca.fit_transform(X)

In [197]:
df_t = pd.read_csv('test.csv')
df_t["risk_factors"] = df_t[[
        "avg_glucose_level", "age", "bmi",
        "hypertension", "heart_disease",
        "smoking_status"
    ]].apply(
        lambda x: \
        0 + (1 if x.avg_glucose_level > 99 else 0) + \
        (1 if x.age > 45 else 0) + (1 if x.bmi > 24.99 else 0) + \
        (1 if x.hypertension == 1 else 0) + \
        (1 if x.heart_disease == 1 else 0) + \
        (1 if x.smoking_status in ["formerly smoked", "smokes"] else 0),
        axis=1
    )

df_t = pd.get_dummies(df_t,columns=['work_type','smoking_status'])
X_t = df.drop(['id','gender','stroke','work_type_Never_worked','ever_married','Residence_type','smoking_status_Unknown','work_type_children'],axis=1)
print(X_t.columns)
X_t=df_t[['work_type_Private','work_type_Govt_job','work_type_Self-employed','risk_factors']]

id = df_t['id']
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_t = sc.fit_transform((X_t))
from keras.utils import np_utils
X_t = X_t.astype('float32')

X_t /= 255

y_p = ann.predict(X_t)
print(len(X_t))
print(len(y_p))

Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'risk_factors', 'work_type_Govt_job', 'work_type_Private',
       'work_type_Self-employed', 'smoking_status_formerly smoked',
       'smoking_status_never smoked', 'smoking_status_smokes'],
      dtype='object')
319/319 [==============================] - 1s 2ms/step
10204
10204


In [198]:
df_t['stroke']=pd.Series([i[0] for i in y_p])
df_t['id']=id
csc = df_t[['id','stroke']]
csc.to_csv('Submission_file_name.csv', index=False , header = 1)


In [56]:
with open('as.csv','w+') as ap:
    for i in y_p:
        ap.write(str(i[0]))
        ap.write('\n')

In [57]:
len(y_p)

10204